In [1]:
import collections
import itertools

In [2]:
testlines = '''kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn'''.splitlines()

In [3]:
with open('day23input.txt') as fp:
    data = fp.read().splitlines()

## Part 1 ##

In [4]:
def get_connections(lines):
    conn = collections.defaultdict(list)
    for line in lines:
        a, b = line.split('-')
        conn[a].append(b)
        conn[b].append(a)
    return conn

In [5]:
def find_triples(conn):
    triples = set()
    for key in conn:
        for a, b in itertools.combinations(conn[key], 2):
            if b in conn[a]:
                triple = tuple(sorted([key, a, b]))
                if triple not in triples:
                    triples.add(triple)
    return triples

In [6]:
def part1(lines):
    conn = get_connections(lines)
    triples = find_triples(conn)
    return sum(1 for a,b,c in triples if a[0] == 't' or  b[0] == 't' or c[0] == 't')
    

In [7]:
assert(7 == part1(testlines))

In [8]:
part1(data)

1378

## Part 2 ##

In [9]:
conn = get_connections(testlines)

In [10]:
def is_all_connected(machines, conn):
    for a,b in itertools.combinations(machines, 2):
        if a in conn[b]:
            continue
        return False
    return True

In [11]:
def max_connected(machine, conn, curr_max):
    sz = len(conn[machine])
    for i in range(sz, curr_max, -1):
        for machines in itertools.combinations([machine] + conn[machine], i):
            if is_all_connected(machines, conn):
                return i, machines
    return 0, []

In [12]:
def part2(lines, thresh):
    conn = get_connections(lines)
    stars = {}
    curr_max = thresh # should be too low
    for machine in conn:
        stars[machine] = max_connected(machine, conn, curr_max)
        if stars[machine][0] > curr_max:
            curr_max = stars[machine][0]
    max_sz = max(stars[machine][0] for machine in stars)
    for machine in stars:
        if stars[machine][0] == max_sz:
            return ','.join(sorted(stars[machine][1]))

In [13]:
assert('co,de,ka,ta' == part2(testlines, 2))

In [14]:
part2(data, 10)

'bs,ey,fq,fy,he,ii,lh,ol,tc,uu,wl,xq,xv'

## Bron-Kerbosch solution ##

from https://www.reddit.com/r/adventofcode/comments/1hkgj5b/comment/m4vwsx9/

In [17]:
network = collections.defaultdict(set)
for line in data:
    a,b = line.rstrip().split('-')
    network[a].add(b)
    network[b].add(a)

In [21]:
def max_cliques(graph):
    # https://en.wikipedia.org/wiki/Bron%E2%80%93Kerbosch_algorithm
    def bron_kerbosch(r, p, x):
        if not p and not x:
            yield ','.join(sorted(r))
        while p:
            v = p.pop()
            yield from bron_kerbosch(r | {v}, p & graph[v], x & graph[v])
            x.add(v)
    yield from bron_kerbosch(set(), set(graph), set())

In [23]:
print(max(max_cliques(network), key=len))

bs,ey,fq,fy,he,ii,lh,ol,tc,uu,wl,xq,xv
